# Web scraping

Another topicthat will be touched during the Big Data course is "scraping": getting unstructured data and turning it into something usable.  The tools available through Python are mature and easy to use.  We'll focus primarily on _web scraping_, where the source data comes in HTML form.

The basic workflow is:

1.  Find the data you want on the web.
2.  Inspect the page you're dealing with, to figure out how to zoom-in towards the content you want.  This will involve some combiation of
    - Looking at the source code of the page (especially if it is simple), and
    - Figuring out the structure of the HTML parse tree.  This step is much easier with a something like __Chrome Developer Tools__.
3.  Write code to get out what you want:
    - If the page is very simple, treat it as a bunch of text => __string manipulation / regular expressions__ in Python.
    - If the page is more complicated (and/or written in good style), we want to use the HTML parse tree => __BeautifulSoup__ in Python.
4.  Make sure it worked!
5.  If your crawling problem is at all non-trivial, you will now have to go back to Step 2 to zoom in further -- or you'll have parsed the URL of a link you want to follow, in which case you'll go back to Step 1 to figure out how to parse what you want from the new target page.

**Example**

As an example, suppose we want to crawl the list of "Available Technologies" being licensed by MIT at http://technology.mit.edu and store their basic info; their associated patents; and the reference counts on their associated patents.

**Step 1**: Let's go to that URL.

- _First try_:  A list of links on the right.  Let's click on a few -- what do we see?  Many are empty, the categories are not obviously mutually exclusive, okay.  Maybe there's a better way.
- _Second try_: Let's just search for all technologies at http://technology.mit.edu/technologies.  Okay, better but it only gives us 50 at a time.  We could just combine the four pages, that's fine.  Let's just click on page 2 to see what happens
- _Third try_: The URL for page 2 is http://technology.mit.edu/technologies?limit=50&offset=50&query=.  That looks like we can just specify a higher limit and offset 0 and get the whole thing.
- _Final answer_: Indeed, http://technology.mit.edu/technologies?limit=1000 has a giant list.

In [ ]:
import urllib2

url = "http://technology.mit.edu/technologies?limit=1000"
raw_page = urllib2.urlopen(url).read()
print(raw_page)

**A quick introduction to HTML and the DOM**

To get started:

- Pull up http://technology.mit.edu/technologies?limit=1000 in Chrome.  
- Open __View->Developer->Developer Tools__.  
- Right click on one of the technology titles, and choose __"Inspect Element"__.

What are we looking at?  Well.. it's this is the structure of the webpage.  Nested _tags_ of different _types_ and having a variety of _attributes_.

**Step 2**: What we learned above:

  - All of the technologies are underneath ("_descendents of_")   `<div class="search" id="nouvant-portfolio-content">`
  - In fact, each of them is in its own `<div class="technology" data-images="true" id="technology_XXXX">`
  
Now we're ready to move on to **Step 3**: We'll use BeautifulSoup to leverage the above to zoom in on the individual technologies and to get links to the pages with detailed info.

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_page)
print soup.prettify()

In [ ]:
parent_div = soup.find('div', attrs={'id': 'nouvant-portfolio-content'}) #Find (at most) *one*
tech_divs = parent_div.find_all('div', attrs={'class':'technology'})  #Find *all*
print len(tech_divs)

** Introduction to CSS selectors**

This pattern -- where you have nested finds, each given by conditions on tag type, id, and class -- is very common.  It's so common that there is a special convenience language for such traversals: [CSS selectors](http://www.w3schools.com/cssref/css_selectors.asp).

BeautifulSoup supports a form of CSS selectors, and this will let us write the above in a more concise and expressive way:
    >    tech_divs = soup.select('div#nouvant-portfolio-content  div.technology')

All selectors work like a 'find_all'.  Some basic building examples of selectors are:

 - _'mytag'_ picks out all tags of type _mytag_.
 - _'#myid'_ picks out all tags whose _id_ is equal to _myid_
 - _'.myclass'_ picks out all tags whose _class_ is equal to _myclass_
 - _'mytag#myid'_ will pick all tags of type _mytag_ **and** _id_ equal to _myid_ (analgously for _'mytag.myclass'_)
 - If _'selector1'_ and _'selector2'_ are two selectors, then there is another selector '_selector1 selector2'_.  It picks out all tags satisfying _selector2_ that are __descendents__(*) of something satisfying _selector1_, i.e., it's like our nested find.
 
 (*) It doesn't have to be a _direct_ descedent.  I.e., it can be a grand-grand-..-grand-child of something satisfying _selector1_.  For direct descendents we'd instead write _'selector1 > selector2'_
 
Let's just explain how this applies to our example:

1.  Let's start with the first half
        >    tech_divs = soup.select('div#nouvant-portfolio-content  div.technology')
        >                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
This picks out all 'div' tags with id 'nouvant-portfolio-content'.
2.  Then the second half
        >    tech_divs = soup.select('div#nouvant-portfolio-content  div.technology')
        >                                                            ^^^^^^^^^^^^^^
This picks out all 'div' tags with class 'technology'.
3.  Finally the whole thing
        >    tech_divs = soup.select('div#nouvant-portfolio-content  div.technology')
        >                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
does exactly the same as our nested find above!

In [ ]:
tech_divs = soup.select('div#nouvant-portfolio-content div.technology')
print len(tech_divs)

Let's check out what we've zoomed in to 

In [ ]:
print tech_divs[0].prettify()

Now we're ready to pull out some key pieces of info:

- The technology's "title" (the text in the `<a>` element)
- The link to follow for more info on the technology (the _href_ attribute of the `<a>`)
- And a short blurb about the text (in the `<span>`)

Let's write some code to extract this.  But before we do, let's discuss what _form_ the output should take: It is often convenient to store data in _key-value_ form (e.g., as a hashtable), in other words to name the bits of data you are collecting.  One big advantage is that this makes it easier to add in extra fields progrssively.

Let's see what the code looks like:

In [ ]:
firsta = tech_divs[0].find('a')
print firsta.text
print firsta['href']

In [ ]:
## 
# We're going to use a "named tuple" to store our key-value data.
# We could also have used a dictionary, with strings as keys.
# Named tuples have some advantages
#  - Better notation, x.field_name instead of x['field_name']
#  - If you change your object structure later and fail to update your
#    code to include the new fields, this will make it easier to find.
#  - They are immutable, preventing certain sorts of bugs.
# .. and some disadvantages:
#  - If you want to augment object structure you need a new type
#    (or to go back and fill your code )
#  - They are immutable.
##
from collections import namedtuple
TechBasic = namedtuple('TechBasic', 'title, url, short')

def td_info(td):
    la = td.select('h2 > a')
    ls = td.select('span')
    if len(la)!=1 or len(ls)!=1:
        print "Uh oh! We did something wrong"
        return None
    return TechBasic (
            title = la[0].text,
            url   = la[0]['href'],
            short = ls[0].text
            )
tech_links=[td_info(td) for td in tech_divs]

print tech_links[0].url

In [ ]:
Patent = namedtuple('Patent', 'name url')
TechDetailed = namedtuple('TechDetailed', 'tech_basic, patents')
def get_tech_details(tech_basic):
    url_base="http://technology.mit.edu/"
    soup = BeautifulSoup( urllib2.urlopen(url_base + tech_basic.url) )
    def patent_info(a):
       return Patent ( 
                name = a.text, 
                url = a['href'] 
                )
    patents = [patent_info(a) for a in soup.select('dd.us_patent_issued a')]
    return TechDetailed ( 
            tech_basic = tech_basic, 
            patents = patents 
            )

tech_basics = map(get_tech_details, tech_links[0:2])  #This takes a list
print tech_basics

**Note**: 
In the last code segment, we only did the first one.  If we try to get them all this way, it'll take a while.  Run the next cell for as long (or not) as you wish, and when you get bored use _Kernel->Interrupt_ to stop it.

The problem is of course that it takes a while to connect to the remote server and fetch the page.  Fortunately, thought it takes a long time it is not actually _computationally expensive_: your computer would be perfectly happy doing this for 20 pages at a time.  The **multiprocessing** package in Python makes it easy to do this kind of (easy) parallelization.

In [ ]:
# Slow version -- takes about 2 minutes to complete
# Uncomment and run it to see
# import time

# start_time = time.time()
# tech_details = map(get_tech_details, tech_links)  #This takes a list
# end_time = time.time()

# print "Done!", end_time-start_time

In [ ]:
# Multi-processor version -- takes a few seconds to complete
import time
from multiprocessing import Pool
workers = Pool(30)  # 30 worker processes

start_time = time.time()
tech_details = workers.map(get_tech_details, tech_links)
end_time = time.time()

print "Done!", end_time-start_time